## 训练阶段

### markdown文本切块

In [ ]:
import os
import re

def split_markdown_by_headers(input_file, output_dir):
    """
    按照大标题（#开头）将Markdown文件切分成多个文件

    Args:
        input_file: 输入的Markdown文件路径
        output_dir: 输出目录
    """
    # 创建输出目录
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"创建目录: {output_dir}")

    # 读取Markdown文件内容
    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read()

    # 使用正则表达式查找所有以#开头的大标题
    # 这里匹配的是以#开头，后面可能有空格，然后是标题文本，直到行尾
    header_pattern = r'^# (.+)$'

    # 查找所有匹配的标题
    headers = re.finditer(header_pattern, content, re.MULTILINE)

    # 记录所有标题的位置和名称
    header_positions = []
    for match in headers:
        header_positions.append((match.start(), match.group(1).strip()))

    # 如果没有找到标题，则将整个文件作为一个部分
    if not header_positions:
        output_file = os.path.join(output_dir, "全部内容.md")
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"保存文件: {output_file}")
        return

    # 处理文件开头到第一个标题之前的内容
    if header_positions[0][0] > 0:
        intro_content = content[:header_positions[0][0]].strip()
        if intro_content:
            output_file = os.path.join(output_dir, "前言.md")
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(intro_content)
            print(f"保存文件: {output_file}")

    # 处理每个标题及其内容
    for i in range(len(header_positions)):
        start_pos = header_positions[i][0]
        title = header_positions[i][1]

        # 确定内容的结束位置
        if i < len(header_positions) - 1:
            end_pos = header_positions[i + 1][0]
        else:
            end_pos = len(content)

        # 提取当前标题的内容
        section_content = content[start_pos:end_pos].strip()

        # 创建安全的文件名
        safe_title = re.sub(r'[\\/*?:"<>|]', '_', title)
        output_file = os.path.join(output_dir, f"{safe_title}.md")

        # 写入文件
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(section_content)

        print(f"保存文件: {output_file}")

if __name__ == "__main__":
    # 输入文件路径
    input_file = r"c:\WorkFlow\E\Code\Python\QNA_Lab\无锡市社会救助帮扶惠民政策清单202420240125_1743663990.526207.md"

    # 输出目录
    output_dir = r"c:\WorkFlow\E\Code\Python\QNA_Lab\mddata"

    # 执行切分
    split_markdown_by_headers(input_file, output_dir)
    print("文件切分完成！")

### jsonl数据转表格展示

In [1]:
import json
import pandas as pd
import re
from pathlib import Path

def extract_think_content(text):
    """从文本中提取<think>标签内的内容"""
    think_pattern = r'<think>(.*?)</think>'
    match = re.search(think_pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return ""

def clean_response(text):
    """清除回答中的<think>标签及其内容"""
    return re.sub(r'<think>.*?</think>\s*', '', text, flags=re.DOTALL).strip()

def jsonl_to_xlsx(input_file, output_file, include_think=False):
    """将JSONL文件转换为XLSX表格"""
    # 读取JSONL文件
    data = []
    with open(input_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:  # 跳过空行
                continue
            try:
                json_obj = json.loads(line)
                messages = json_obj.get('messages', [])
                if len(messages) >= 2:
                    question = messages[0].get('content', '')
                    full_answer = messages[1].get('content', '')

                    # 提取思考过程和清理后的回答
                    think_content = extract_think_content(full_answer)
                    clean_answer = clean_response(full_answer)

                    if include_think:
                        data.append({
                            '问题': question,
                            '答案': clean_answer,
                            '思考过程': think_content
                        })
                    else:
                        data.append({
                            '问题': question,
                            '答案': clean_answer
                        })
            except json.JSONDecodeError:
                print(f"警告: 无法解析JSON行: {line[:100]}...")
                continue

    # 创建DataFrame并保存为XLSX
    df = pd.DataFrame(data)
    df.to_excel(output_file, index=False)
    print(f"已成功将数据保存到 {output_file}")

if __name__ == "__main__":
    # 配置参数
    input_file = r"C:\WorkFlow\E\Code\Python\QNA_Lab\data\帮扶-1744006595791-sharegpt-2025-04-08.jsonl"
    output_file = r"C:\WorkFlow\E\Code\Python\QNA_Lab\data\帮扶数据.xlsx"

    # 是否包含思考过程的开关参数
    include_think = True  # 设置为 True 包含思考过程，False 则不包含

    # 执行转换
    print(f"开始处理文件: {input_file}")
    print(f"是否包含思考过程: {'是' if include_think else '否'}")

    jsonl_to_xlsx(input_file, output_file, include_think)

    print("处理完成!")

开始处理文件: C:\WorkFlow\E\Code\Python\QNA_Lab\data\帮扶-1744006595791-sharegpt-2025-04-08.jsonl
是否包含思考过程: 是
已成功将数据保存到 C:\WorkFlow\E\Code\Python\QNA_Lab\data\帮扶数据.xlsx
处理完成!


## 输出阶段

### json_str 转 json_obj
 - 注意：Python展示json默认是单引号（python dict），而json的标准格式是双引号。 

In [1]:
import json
import re
from typing import Dict

def robust_json_parser(raw_text: str) -> Dict:
    # 清理非法前缀和杂项字符
    cleaned_text = re.sub(r'^[^{]*{', '{', raw_text, flags=re.DOTALL)
    # 处理中文冒号等非标符号
    cleaned_text = cleaned_text.replace('：', ':')
    # 移除尾部杂字符
    cleaned_text = re.sub(r'}[^}]*$', '}', cleaned_text)
    return json.loads(cleaned_text)

if __name__ == "__main__":
    # 测试数据
    test_json = """{
        "Project_01" : {
        "No.": 1,
        "projectName": "项目名称",
        "budgetInTenThousand": 50.8,
        "projectDetails": "具体内容."
        }
    }"""

    # 执行测试
    try:
        result = robust_json_parser(test_json)
        print("解析成功:")
        print(result)
    except Exception as e:
        print(f"解析失败: {str(e)}")

解析成功:
{'Project_01': {'No.': 1, 'projectName': '项目名称', 'budgetInTenThousand': 50.8, 'projectDetails': '具体内容.'}}
